## Import Statements

In [17]:
import numpy as np
import pypico
import time as t

## Load data (WMAP, PICO, SNe Ia)

In [3]:
#Load the wmap satellite data
wmap=np.loadtxt('wmap_tt_spectrum_9yr_v5.txt')
multipole = wmap[:,0]; power = wmap[:,1]; errPower = wmap[:,2]

#Load the pico training data
pico = pypico.load_pico("jcset_py3.dat")

#Load the SNe Ia data
sn_z,sn_dm,sn_dm_err = np.loadtxt("SCPUnion2.1_mu_vs_z.txt",delimiter="\t",skiprows=5, usecols = (1,2,3),unpack=True)

## Posterior Probability Function for WMAP

In [14]:
def get_PICO_spectrum(pars):
    """
    Function to evaluate the CAMB model emulator for a given set of parameters. 
    Much faster than the full CAMB model
    """
    
    H0, ombh2, omch2, omk, tau, As, ns, alpha = pars #Unpack model parameters
    
    input_dict = {"As": As,"ns": ns,"tau": tau,"ombh2":ombh2,"omch2":omch2,"H0":H0,"omk":omk}
    #force=True will make pico evaluate the emulator even if we are outside the training data domain
    #Alternatively could catch the CantUsePICO error and evaluate CAMB, but force should work fine
    output_dict = pico.get(**input_dict, force=True)
    tt = output_dict['dl_TT']
    
    return tt

In [15]:
def get_cov_model(err, alpha):
    """
    Evaluate the covariance matrix using a model where adjacent points have a correlation 
    scaled by the parameter alpha
    
    0.16 seconds with vectorization, 0.36 seconds with for loops
    """
    N = len(err)
    C = np.zeros([N,N])
    #Compute each element in the covariance matrix
    err_shift_1 = np.roll(err,-1) #Get the error shifted by one, to compute k=1 correlation terms
    #Compute the diagonal and k=1 terms
    diag_terms = err**2
    diag_k1_terms = alpha*np.abs(err[0:-1]*err_shift_1[0:-1]) #Should I take the absolute value?
    #Cast the terms into matrix form and combine to get the final covariance matrix
    C_diag = np.diag(np.array(diag_terms))
    C_diag_k1 = np.diag(np.array(diag_k1_terms), k=1)
    C_diag_km1 = np.diag(np.array(diag_k1_terms), k=-1)
    C = C_diag + C_diag_k1 + C_diag_km1
    
    return C

def log_likelihood_WMAP(theta, multipole, p_data, err):
    """
    Evaluate the chi-sq metric of a PICO fit with a neighbour-correlation model, 
    given a set of model parameters stored in the array theta
    
    Return the log likelihood probability, which is equal to -chi_sq
    """
    
    #Could add a control switch to evaluate the correlated or uncorrelated chi-sq
    
    alpha = theta[7] #Get the covariance scaling parameter
    
    #Get p_model using get_PICO_spectrum(theta)
    pico_tt = get_PICO_spectrum(theta) #Note that the last element of theta isn't used, but it shouldnt cause a problem because I read the theta values out properly in get_PICO_spectrum
    p_model = pico_tt[2:len(multipole)+2]
    
    A = np.array([p_data - p_model])
    At = np.transpose(A)
    C_inv = np.linalg.inv(get_cov_model(err, alpha))
    
    chi_sq_mat = At*C_inv*A #Evaluate the matrix of chi-squared terms
    chi_sq = sum(sum(chi_sq_mat)) #Evaluate the chi-squared metric by summing all terms
    
    return -chi_sq

def log_prior_WMAP(theta):
    """
    Evaluate the log prior probability function given model parameters
    
    Return 0.0 if the parameters fall within our constraints, else return -np.inf
    """
    H0, ombh2, omch2, omk, tau, As, ns, alpha = theta #Unpack model parameters
    
    #Convert units of Omega params
    h = H0/100
    Omb = ombh2/(h**2)
    Omde = omch2/(h**2)
    
    #Check that the params are allowed by our physical constraints
    if 0. <= Omb <= 1. and 0. < Omde < 1. and -1.<=alpha<=1.:
        return 0.0 # the constant doesn't matter since MCMCs only care about *ratios* of probabilities
    return -np.inf # log(0) = -inf

def log_post_WMAP(theta, multipole, p_data, err):
    """
    Evaluate the log posterior probability function given WMAP data and model parameters
    """
    lp = log_prior_WMAP(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood_WMAP(theta, multipole, p_data, err)

In [22]:
#Test the log_post_WMAP function using a dummy set of parameter values
pars_initialGuess=np.asarray([70,0.02,0.1,0.0,0.05,2e-9,0.97,0.1])

t1 = t.time()
p_log_post = log_post_WMAP(pars_initialGuess, multipole, power, errPower)
t2 = t.time()
print(t2-t1,'seconds per log_post_WMAP call')
print('Log Posterior value for the given params:',p_log_post)

0.47792816162109375 seconds per log_post_WMAP call
Log Posterior value for the given params: -1414.823194085791


## MCMC Helper Functions

MCMC Animation

In [1]:
def plot_SNe_sample(x_data, flat_samples, ind):
    #Get the parameters, create the corresponding cosmo model, and evaluate mu
    sample = flat_samples[ind]
    H0, Om0, Ode0 = sample
    cosmo = LambdaCDM(H0=H0, Om0=Om0, Ode0=Ode0) 
    y_model = mu_func(x_data, cosmo)
    #Plot the fit for these parameters
    plt.plot(x_data, y_model, alpha=0.01, color='red',zorder=2)
    #Save the figure to add to the animation
    fig_name = 'frame'+str(ind)+'.png'
    plt.savefig(fig_name)
    
    return fig_name

def plot_WMAP_sample(x_data, flat_samples, ind):
    #Get the parameters and evaluate the corresponding PICO model
    sample = flat_samples[ind]
    y_model = get_PICO_spectrum(sample)
    y_model = y_model[2:len(x_data)+2]
    #Plot the fit for these parameters
    plt.plot(x_data, y_model, alpha=0.01, color='red',zorder=2)
    #Save the figure to add to the animation
    fig_name = 'frame'+str(ind)+'.png'
    plt.savefig(fig_name)
    
    return fig_name
    
def write_animation(fig_name_list, filename):
    """
    Take a series of .png frames and animate them into a .gif
    """
    # build gif from the frames in the directory
    with imageio.get_writer(filename, mode='I') as writer:
        for fig_name in fig_name_list:
            image = imageio.imread(fig_name)
            writer.append_data(image)

    # Clear the files from the directory files
    for fig_name in set(fig_name_list):
        os.remove(fig_name)
        
    print('Animation saved as ', filename)

def MCMC_animation(sampler, x_data, y_data, y_err, dataset, filename, N_samples):
    """
    Create an animation of the first N_samples of the MCMC fit
    """
    
    #Plot the original data
    plt.figure(figsize=(7,7))
    if dataset == 'SNe':
        plt.plot(x_data, y_data,'.k')
        #plt.errorbar(x_data, y_data, yerr=y_err, linestyle = 'None', fmt='.k',mec='black',mfc='black',ecolor='grey',zorder=1)
        plt.xlabel(r'$z$')
        plt.ylabel(r'$m-M (Mag)$')
        plt.xscale('log')
        plt.title('SCP Union 2.1 SNe Ia Data')
    if dataset =='WMAP':
        plt.plot(multipole,power)
        #plt.errorbar(wmap[:,0],wmap[:,1],wmap[:,2],fmt='*')
        plt.xlabel('Multipole Moment')
        plt.ylabel('Power Spectrum')
        plt.title('WMAP Satellite 9-year CMB Data')
        plt.legend()
    
    #Get the samples to plot
    flat_samples = sampler.get_chain(flat=True)[:N_samples]
    
    #Plot each sample and save the plot frame as a .png
    fig_name_list = []
    for ind in range(N_samples):
        if dataset == 'SNe':
            fig_name = plot_SNe_sample(x_data, flat_samples, ind)
        if dataset == 'WMAP':
            fig_name = plot_WMAP_sample(x_data, flat_samples, ind)
        #Store the frame filename
        fig_name_list.append(fig_name)
        
    #Collect the .png frames and save them as a .gif animation
    write_animation(fig_name_list, filename)

## WMAP MCMC Fitting

In [4]:
#Check tau_f to tune MCMC, show corner plots of results
#Plot the posterior fit with the data
#Convert omega*h^2 values to omega

WMAP MCMC Animation

In [ ]:
filename = 'WMAP_MCMC_animation.gif'
dataset = 'WMAP'
N_samples = 100
#Create animation of MCMC fitting
MCMC_animation(sampler_WMAP, multipole, power, errPower, dataset, filename, N_samples)

## Posterior Probability Function for SNe Ia

## SNe Ia MCMC Fitting

In [5]:
#Check tau_f to tune MCMC, show corner plots of results
#Plot the posterior fit with the data

#Calculate posterior for omega_k = 1 - omega_m - omega_de

SNe MCMC Animation

In [ ]:
filename = 'SNe_MCMC_animation.gif'
dataset = 'SNe'
N_samples = 100
#Create animation of MCMC fitting
MCMC_animation(sampler_SNe, sn_z, sn_dm, sn_dm_err, dataset, filename, N_samples)

## Overlay comparison of WMAP and SNe Ia Posterior distributions

In [7]:
#Overlay the posteriors for the omegas to compare between the two measurements
#Maybe include best fit values from Planck?